In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.linear_model import LinearRegression
import scipy.stats as ss

In [2]:
df = pd.read_csv('C:/Users/owkr9001/Downloads/dane do cwiczenia/dane.csv', header=0, sep='[;]', engine='python', 
                 names=('client_id','action_time', 'pagePath', 'medium', 'eventAction', 'date'))
df.client_id = df.client_id.str[1:]
df.date = df.date.str[:-1]
df.head()

,client_id,action_time,pagePath,medium,eventAction,date
0,d0a213b0718cd0e10475ce11d893c3cc,2020-07-29T15:38:27Z,"/lekcja/biologia,75f95eb9a371db8c2ea854da85219502",organic,pageview,29.07.2020
1,4416f6ec763cc9e65ca17482d3960971,2020-07-29T21:03:06Z,/,(none),pageview,29.07.2020
2,c3d6e02ad3e2a781dba7c48a4d354199,2020-07-29T07:51:28Z,/o-portalu,banner,pageview,29.07.2020
3,3bd4feb129163c7889a70d3d12c6e82d,2020-07-29T12:23:00Z,"/lekcja/geografia,12901cca29551ad7d556962231a1...",organic,pageview,29.07.2020
4,3d90c06c313a2120ce98ff14b6581570,2020-07-29T12:27:30Z,"/lekcja/jezyk-polski,c106aef9f68b2b03b8afe7d89...",organic,pageview,29.07.2020


In [3]:
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df['time'] = pd.to_datetime(df['action_time']).dt.time
df.drop(['action_time'], axis='columns', inplace=True)
df.head()

,client_id,pagePath,medium,eventAction,date,time
0,d0a213b0718cd0e10475ce11d893c3cc,"/lekcja/biologia,75f95eb9a371db8c2ea854da85219502",organic,pageview,2020-07-29,15:38:27
1,4416f6ec763cc9e65ca17482d3960971,/,(none),pageview,2020-07-29,21:03:06
2,c3d6e02ad3e2a781dba7c48a4d354199,/o-portalu,banner,pageview,2020-07-29,07:51:28
3,3bd4feb129163c7889a70d3d12c6e82d,"/lekcja/geografia,12901cca29551ad7d556962231a1...",organic,pageview,2020-07-29,12:23:00
4,3d90c06c313a2120ce98ff14b6581570,"/lekcja/jezyk-polski,c106aef9f68b2b03b8afe7d89...",organic,pageview,2020-07-29,12:27:30


In [4]:
df["medium"] = df["medium"].astype('category')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466822 entries, 0 to 466821
Data columns (total 6 columns):
client_id      466822 non-null object
pagePath       466822 non-null object
medium         466822 non-null category
eventAction    466822 non-null object
date           466822 non-null datetime64[ns]
time           466822 non-null object
dtypes: category(1), datetime64[ns](1), object(4)
memory usage: 18.3+ MB


In [6]:
df.eventAction.describe()

count       466822
unique           1
top       pageview
freq        466822
Name: eventAction, dtype: object

In [7]:
df.drop(['eventAction'], axis='columns', inplace=True)

In [8]:
df.describe()

,client_id,pagePath,medium,date,time
count,466822,466822,466822,466822,466822
unique,230193,2549,10,37,70553
top,64e4c7ca28927cfd62a503e100afe752,/,organic,2020-08-21 00:00:00,10:08:41
freq,855,67224,260306,25938,26
first,NaN,NaN,NaN,2020-07-29 00:00:00,NaN
last,NaN,NaN,NaN,2020-10-09 00:00:00,NaN


In [9]:
df.describe(include=np.object)

,client_id,pagePath,time
count,466822,466822,466822
unique,230193,2549,70553
top,64e4c7ca28927cfd62a503e100afe752,/,10:08:41
freq,855,67224,26


In [11]:
df['medium'].value_counts()

organic      260306
(none)        67090
cpc           50864
banner        46942
referral      30169
email         11243
post            102
link             74
webpush          31
(not set)         1
Name: medium, dtype: int64

In [12]:
df["client_id"].nunique()

230193

In [13]:
df1 = df['medium'].value_counts().plot(kind="bar")

In [14]:
df['pageCat'] = df['pagePath']
df['pageMainCat'] = df['pagePath']

In [15]:
categories = {
    '/lekcja/biologia': 'lekcja-biologia', 
    '/lekcja/geografia': 'lekcja-geografia',
    '/lekcja/historia': 'lekcja-historia', 
    '/lekcja/matematyka': 'lekcja-matematyka',
    '/lekcja/jezyk-polski': 'lekcja-jezyk-polski',
    '/lekcja/jezyk-niemiecki': 'lekcja-jezyk-niemiecki',
    '/lekcja/jezyk-angielski': 'lekcja-jezyk-angielski',
    
    '/kategorie/biologia': 'kategorie-biologia',
    '/kategorie/geografia': 'kategorie-geografia',
    '/kategorie/historia': 'kategorie-historia', 
    '/kategorie/matematyka': 'kategorie-matematyka',
    '/kategorie/jezyk-polski': 'kategorie-jezyk-polski',
    '/kategorie/jezyk-niemiecki': 'kategorie-jezyk-niemiecki',
    '/kategorie/jezyk-angielski': 'kategorie-jezyk-angielski',
   
    '/fiszki-i-quizy/biologia': 'fiszki-i-quizy-biologia',
    '/fiszki-i-quizy/geografia': 'fiszki-i-quizy-geografia',
    '/fiszki-i-quizy/historia': 'fiszki-i-quizy-historia', 
    '/fiszki-i-quizy/matematyka': 'fiszki-i-quizy-matematyka',
    '/fiszki-i-quizy/jezyk-polski': 'fiszki-i-quizy-jezyk-polski',
    '/fiszki-i-quizy/jezyk-niemiecki': 'fiszki-i-quizy-jezyk-niemiecki',
    '/fiszki-i-quizy/jezyk-angielski': 'fiszki-i-quizy-jezyk-angielski',
    
    '/sprawdz-sie/biologia': 'sprawdz-sie-biologia',
    '/sprawdz-sie/geografia': 'sprawdz-sie-geografia',
    '/sprawdz-sie/historia': 'sprawdz-sie-historia', 
    '/sprawdz-sie/matematyka': 'sprawdz-sie-matematyka',
    '/sprawdz-sie/jezyk-polski': 'sprawdz-sie-jezyk-polski',
    '/sprawdz-sie/jezyk-niemiecki': 'sprawdz-sie-jezyk-niemiecki',
    '/sprawdz-sie/jezyk-angielski': 'sprawdz-sie-jezyk-angielski',    
    
    '/quiz/': 'quiz',
    '/fiszki/': 'fiszki',
    '/szukaj/': 'szukaj',
    'lesson/preview': 'lesson/preview',
    '/o-portalu': 'o-portalu',
    'gref': 'gref',
    '?error': 'error'
    
}
    
for key, value in categories.items():
    df['pageCat'] = df['pageCat'].astype(str).apply(lambda X:  value if key in X else X)
    
owi_set = set(categories.values())
df['pageCat'] = df['pageCat'].apply(lambda x: x if x in owi_set else 'Others')    

In [16]:
main_categories = {
    '/lekcja/biologia': 'lekcja', 
    '/lekcja/geografia': 'lekcja',
    '/lekcja/historia': 'lekcja', 
    '/lekcja/matematyka': 'lekcja',
    '/lekcja/jezyk-polski': 'lekcja',
    '/lekcja/jezyk-niemiecki': 'lekcja',
    '/lekcja/jezyk-angielski': 'lekcja',
    
    '/kategorie/biologia': 'kategorie',
    '/kategorie/geografia': 'kategorie',
    '/kategorie/historia': 'kategorie', 
    '/kategorie/matematyka': 'kategorie',
    '/kategorie/jezyk-polski': 'kategorie',
    '/kategorie/jezyk-niemiecki': 'kategorie',
    '/kategorie/jezyk-angielski': 'kategorie',
   
    '/fiszki-i-quizy/biologia': 'fiszki-i-quizy',
    '/fiszki-i-quizy/geografia': 'fiszki-i-quizy',
    '/fiszki-i-quizy/historia': 'fiszki-i-quizy', 
    '/fiszki-i-quizy/matematyka': 'fiszki-i-quizy',
    '/fiszki-i-quizy/jezyk-polski': 'fiszki-i-quizy',
    '/fiszki-i-quizy/jezyk-niemiecki': 'fiszki-i-quizy',
    '/fiszki-i-quizy/jezyk-angielski': 'fiszki-i-quizy',
    
    '/sprawdz-sie/biologia': 'sprawdz-sie',
    '/sprawdz-sie/geografia': 'sprawdz-sie',
    '/sprawdz-sie/historia': 'sprawdz-sie', 
    '/sprawdz-sie/matematyka': 'sprawdz-sie',
    '/sprawdz-sie/jezyk-polski': 'sprawdz-sie',
    '/sprawdz-sie/jezyk-niemiecki': 'sprawdz-sie',
    '/sprawdz-sie/jezyk-angielski': 'sprawdz-sie',    
    
    '/quiz/': 'quiz',
    '/fiszki/': 'fiszki',
    '/szukaj/': 'szukaj',
    'lesson/preview': 'lesson/preview',
    '/o-portalu': 'o-portalu',
    'gref': 'gref',
    '?error': 'error'
    
}
    
for key, value in main_categories.items():
    df['pageMainCat'] = df['pageMainCat'].astype(str).apply(lambda X:  value if key in X else X) 
    
owi_set = set(main_categories.values())
df['pageMainCat'] = df['pageMainCat'].apply(lambda x: x if x in owi_set else 'Others')  

In [17]:
df['pageCat'].value_counts()

Others                            78222
lekcja-jezyk-polski               72942
quiz                              59599
lekcja-geografia                  55764
lekcja-biologia                   43633
fiszki                            42901
lekcja-historia                   38264
o-portalu                         30812
lekcja-matematyka                 20117
lekcja-jezyk-angielski             6596
kategorie-jezyk-polski             4153
kategorie-biologia                 2751
kategorie-geografia                2646
kategorie-matematyka               2347
kategorie-historia                 1398
kategorie-jezyk-angielski          1240
lekcja-jezyk-niemiecki             1057
lesson/preview                      677
szukaj                              556
kategorie-jezyk-niemiecki           357
fiszki-i-quizy-geografia            315
fiszki-i-quizy-biologia             163
fiszki-i-quizy-historia              53
fiszki-i-quizy-matematyka            46
error                                39


In [18]:
df["pageCat"] = df["pageCat"].astype('category')
df["pageMainCat"] = df["pageMainCat"].astype('category')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466822 entries, 0 to 466821
Data columns (total 7 columns):
client_id      466822 non-null object
pagePath       466822 non-null object
medium         466822 non-null category
date           466822 non-null datetime64[ns]
time           466822 non-null object
pageCat        466822 non-null category
pageMainCat    466822 non-null category
dtypes: category(3), datetime64[ns](1), object(3)
memory usage: 15.6+ MB


In [20]:
df

,client_id,pagePath,medium,date,time,pageCat,pageMainCat
0,d0a213b0718cd0e10475ce11d893c3cc,"/lekcja/biologia,75f95eb9a371db8c2ea854da85219502",organic,2020-07-29,15:38:27,lekcja-biologia,lekcja
1,4416f6ec763cc9e65ca17482d3960971,/,(none),2020-07-29,21:03:06,Others,Others
2,c3d6e02ad3e2a781dba7c48a4d354199,/o-portalu,banner,2020-07-29,07:51:28,o-portalu,o-portalu
3,3bd4feb129163c7889a70d3d12c6e82d,"/lekcja/geografia,12901cca29551ad7d556962231a1...",organic,2020-07-29,12:23:00,lekcja-geografia,lekcja
4,3d90c06c313a2120ce98ff14b6581570,"/lekcja/jezyk-polski,c106aef9f68b2b03b8afe7d89...",organic,2020-07-29,12:27:30,lekcja-jezyk-polski,lekcja
...,...,...,...,...,...,...,...
466817,4aeb67e947bf0f68727da4531cb2e730,/,cpc,2020-09-03,20:23:49,Others,Others
466818,85298782f748e544275ea0ba55767d39,"/lekcja/biologia,43eb0ab6190bc7e8acf5d294119e22c9",organic,2020-09-12,12:44:06,lekcja-biologia,lekcja
466819,fcb3a6846e37f653bd39b2f1cc9cd338,"/lekcja/historia,a7020fcc21842131d9a211d1e9c558fa",organic,2020-09-18,13:58:39,lekcja-historia,lekcja
466820,fcb3a6846e37f653bd39b2f1cc9cd338,"/lekcja/historia,a7020fcc21842131d9a211d1e9c558fa",cpc,2020-09-18,14:24:09,lekcja-historia,lekcja


In [21]:
def cramers_v(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher,
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

In [22]:
confusion_matrix = pd.crosstab(df["date"], df["time"]).as_matrix()
cramers_v(confusion_matrix)

C:\Users\owkr9001\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


0.08932638793980605

In [23]:
df['target'] = df['medium'].astype(str).apply(lambda X:  value if key in X else X)

In [24]:
print(df.isnull().values.sum())

0


In [25]:
labels = df['medium'].astype('category').cat.categories.tolist()
rep_map = {'medium' : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}

print(rep_map)

{'medium': {'(none)': 1, '(not set)': 2, 'banner': 3, 'cpc': 4, 'email': 5, 'link': 6, 'organic': 7, 'post': 8, 'referral': 9, 'webpush': 10}}


In [26]:
df['pageCat_encoded'] = df['pageCat'].cat.codes

In [27]:
df

,client_id,pagePath,medium,date,time,pageCat,pageMainCat,target,pageCat_encoded
0,d0a213b0718cd0e10475ce11d893c3cc,"/lekcja/biologia,75f95eb9a371db8c2ea854da85219502",organic,2020-07-29,15:38:27,lekcja-biologia,lekcja,organic,18
1,4416f6ec763cc9e65ca17482d3960971,/,(none),2020-07-29,21:03:06,Others,Others,(none),0
2,c3d6e02ad3e2a781dba7c48a4d354199,/o-portalu,banner,2020-07-29,07:51:28,o-portalu,o-portalu,banner,26
3,3bd4feb129163c7889a70d3d12c6e82d,"/lekcja/geografia,12901cca29551ad7d556962231a1...",organic,2020-07-29,12:23:00,lekcja-geografia,lekcja,organic,19
4,3d90c06c313a2120ce98ff14b6581570,"/lekcja/jezyk-polski,c106aef9f68b2b03b8afe7d89...",organic,2020-07-29,12:27:30,lekcja-jezyk-polski,lekcja,organic,23
...,...,...,...,...,...,...,...,...,...
466817,4aeb67e947bf0f68727da4531cb2e730,/,cpc,2020-09-03,20:23:49,Others,Others,cpc,0
466818,85298782f748e544275ea0ba55767d39,"/lekcja/biologia,43eb0ab6190bc7e8acf5d294119e22c9",organic,2020-09-12,12:44:06,lekcja-biologia,lekcja,organic,18
466819,fcb3a6846e37f653bd39b2f1cc9cd338,"/lekcja/historia,a7020fcc21842131d9a211d1e9c558fa",organic,2020-09-18,13:58:39,lekcja-historia,lekcja,organic,20
466820,fcb3a6846e37f653bd39b2f1cc9cd338,"/lekcja/historia,a7020fcc21842131d9a211d1e9c558fa",cpc,2020-09-18,14:24:09,lekcja-historia,lekcja,cpc,20
